In [1]:
import torch
import random
import torch.nn as nn
from tqdm import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
def teacher_and_student_loss_fn(t_out_4, s_out_4, labels, temperature = 3, alpha = 0.7):
    cce_l = nn.CrossEntropyLoss()(s_out_4, labels.view(-1))
    s_out_4 = nn.LogSoftmax()(s_out_4/temperature)
    t_out_4 = torch.softmax(t_out_4/temperature, dim=-1)
    kde_loss = nn.KLDivLoss()(s_out_4, t_out_4)
    
    return kde_loss*(alpha * temperature* temperature) + cce_l * (1 - alpha)

In [4]:
def accuracy_fn(out, ground_truth):
    prediction = torch.argmax(out, dim=-1)
    return ((prediction.reshape(-1, 1) == ground_truth)*1.0).mean()